<a href="https://colab.research.google.com/github/7PAM2015-0509-2023-Group-20/Kaggle-Challenge-20/blob/main/Group_project_Kaggle_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.4-py2.py3-none-any.whl size=265375577 sha256=164c8053044ae7f427c50b1a964577c12ddbcfae0f947116c94e615da01d059e
  Stored in directory: /root/.cache/pip/wheels/4d/a6/47/8bfeb1026fd65cb8630beb74d8e3bec844f572cf4f336fdd56
Successfully built h2o


Importing libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")


Importing libraries for H2O AutoML and NetworkX

In [3]:
import h2o
from h2o.automl import H2OAutoML
import networkx as nx

In [4]:
# Set styles
sns.set(style="whitegrid", font_scale=1.2)
pd.set_option("display.max_columns", None)

Load datasets

In [5]:
train_data = pd.read_csv("https://raw.githubusercontent.com/7PAM2015-0509-2023-Group-20/Kaggle-Challenge-20/main/train.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/7PAM2015-0509-2023-Group-20/Kaggle-Challenge-20/main/test.csv")

EDA

In [6]:
print(f"Train dataset shape: {train_data.shape}")
print(f"Test dataset shape: {test_data.shape}")

Train dataset shape: (8693, 14)
Test dataset shape: (4277, 13)


In [7]:
print(f"Duplicate rows in train dataset: {train_data.duplicated().sum()}")
print(f"Duplicate rows in test dataset: {test_data.duplicated().sum()}")

Duplicate rows in train dataset: 0
Duplicate rows in test dataset: 0


In [7]:
print("Data types in training data:")
print(train_data.dtypes)

In [8]:
print("Data types in testing data:")
print(test_data.dtypes)

Data types in testing data:
PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
dtype: object


Handle missing values

In [9]:
missing_train = train_data.isnull().sum()[train_data.isnull().sum() > 0]
missing_test = test_data.isnull().sum()[test_data.isnull().sum() > 0]
print("Missing values in train data:")
print(missing_train)
print("Missing values in test data:")
print(missing_test)

Missing values in train data:
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
dtype: int64
Missing values in test data:
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64


Feature engineering

In [10]:
def create_new_features(df):
    # PassengerId-based features
    df['Group'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
    df['Member'] = df['PassengerId'].apply(lambda x: x.split('_')[1])
    df['Solo_Travel'] = df['Group'].map(df['Group'].value_counts()) == 1
    df['Group_Size'] = df['Group'].map(df['Group'].value_counts())

    # Cabin-based features
    df['Cabin'].fillna('nan/nan/nan', inplace=True)
    df['Cabin_Deck'] = df['Cabin'].apply(lambda x: x.split('/')[0])
    df['Cabin_Num'] = df['Cabin'].apply(lambda x: x.split('/')[1])
    df['Cabin_Side'] = df['Cabin'].apply(lambda x: x.split('/')[2])
    df[['Cabin_Deck', 'Cabin_Num', 'Cabin_Side']] = df[['Cabin_Deck', 'Cabin_Num', 'Cabin_Side']].replace('nan', np.nan)
    df['Cabin_Deck'].fillna(df['Cabin_Deck'].mode()[0], inplace=True)
    df['Cabin_Side'].fillna(df['Cabin_Side'].mode()[0], inplace=True)
    df['Cabin_Num'] = df['Cabin_Num'].astype(float).fillna(df['Cabin_Num'].median())

    df['Cabin_Region'] = pd.cut(df['Cabin_Num'], bins=[0, 300, 600, 900, 1200, 1500, np.inf], labels=[1, 2, 3, 4, 5, 6])

    # Age-based features
    df['Age_Group'] = pd.cut(df['Age'], bins=[-np.inf, 12, 18, 25, 32, 50, np.inf], labels=['0-12', '13-18', '19-25', '26-32', '33-50', '50+'])

    # Expenditure features
    df['Total_Expenditure'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['No_Spending'] = df['Total_Expenditure'] == 0

In [11]:
create_new_features(train_data)
create_new_features(test_data)

Fill missing values

In [12]:
cat_features = train_data.select_dtypes(include=['object', 'bool']).columns.to_list()
num_features = train_data.select_dtypes(include=['int', 'float']).columns.to_list()
cat_features.remove('Transported')